# Massively parallel Transition Path Sampling

## Notebook 1: Run TPS simulation

This is the first of a series of example notebooks on massively parallel transition path sampling. We will use multiple samplers (each generating its own Markov chain) that are all steered by one central reaction coordinate model. This has the benefit that the central model learns from all chains/samplers, which is more time efficient since we can run all samplers at the same time (e.g. on an HPC cluster). And since the chains will diverge from each other the central model will also see multiple reaction mechanisms at the same time, i.e. it will learn all of them. Since all chains have equal weight you can still easily calculate ensemble averages over the transition path ensemble by weighting each Monte carlo state in each chain with equal weight. Note, that to start the sampling we do need an initial transition for each sampler/Markov chain (as opposed to TPS from equilibrium points). If you use different initial transitions for each sampler this is actually a benefit as it enables you to estimate if the ensemble is converged by comparing the chains to each other (which you will see in the analysis notebook).

In this notebook we will use capped alanine dipetide as our example molecule and look at the transition between $C7_{eq}$ and $\alpha_R$ states. We will use the locally running `GmxEngine` and `PyTrajectoryFunctionWrapper` classes (such that you can run it on your workstation), but you can easily perform a massively parallel TPS on a HPC cluster running SLURM by using the `SlurmGmxEngine` and `SlurmTrajectoryFunctionWrapper` classes instead. However, in that case you will probably want to use a larger (and more interessting) system than capped alanine dipeptide :)

**This notebook should be run on a multi-core workstation preferably with a GPU**, otherwise you will have a very long coffee break and a very hot laptop.

**Required knowledge/recommended reading:** This notebooks assumes some familarity with the `asyncmd` (namely the [gromacs] engine and TrajectoryFunctionWrapper classes). Please see the example notebooks in `asyncmd` for an introduction.

In [1]:
%matplotlib inline

In [2]:
import os
import asyncio
import numpy as np
import matplotlib.pyplot as plt
#import mdtraj as mdt
import MDAnalysis as mda
import asyncmd
import asyncmd.gromacs as asyncgmx
from asyncmd import Trajectory
import aimmd
import aimmd.distributed as aimmdd

/home/tb/hejung/.conda/envs/aimmd_distributed_devel/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not initialize SLURM cluster handling. If you are sure SLURM (sinfo/sacct/etc) is available try calling `asyncmd.config.set_slurm_settings()` with the appropriate arguments.
Tensorflow/Keras not available


In [3]:
# setup working directory

scratch_dir = "/homeloc/scratch/aimmd_distributed/"
#scratch_dir = "."

workdir = os.path.join(scratch_dir, "TransitionPathSampling_ala")

if not os.path.isdir(workdir):
    os.mkdir(workdir)

In [4]:
# setup logging
# executing this file sets the variable LOGCONFIG, which is a dictionary of logging presets 
%run ../resources/logconf.py

In [5]:
# have a look at the default logging level (the level used for the root logger)
print(LOGCONFIG["loggers"][""])
# have a look at the logger for aimmd
print(LOGCONFIG["loggers"]["aimmd"])
# and have a look at the log-level for the filehandler
print(LOGCONFIG["handlers"]["stdf"])
# the last two should both be `INFO`

{'level': 'WARN', 'handlers': ['stdf', 'warnout']}
{'level': 'INFO'}
{'class': 'logging.FileHandler', 'level': 'INFO', 'mode': 'w', 'filename': 'simulation.log', 'formatter': 'standardFormatter'}


In [6]:
# OPTIONAL: more logging to file
level = "INFO"
LOGCONFIG["handlers"]["stdf"]["level"] = level
LOGCONFIG["loggers"]["aimmd"]["level"] = level
LOGCONFIG["loggers"]["asyncmd"] = {"level": level}

In [7]:
# you can either modify single values or use it as is to get the same setup as in the OPS default logging config file
# you could e.g. do LOGCONF['handlers']['stdf']['filename'] = new_name to change the filename of the log
# the default is to create 'simulation.log' and 'initialization.log' in the current working directory
import logging.config
LOGCONFIG["handlers"]["stdf"]["filename"] = os.path.join(workdir, "simulation_pathsampling.log")
LOGCONFIG["handlers"]["initf"]["filename"] = os.path.join(workdir, "initlog_pathsampling.log")
logging.config.dictConfig(LOGCONFIG)

# now the actual setup

In [8]:
n_samplers = 5  # results in 2*n_samplers gmx engines

In [9]:
storage = aimmd.Storage(os.path.join(workdir, "storage.h5"))

In [10]:
# Define the engine(s) for the PathMovers
# (they will all be the same)
gro = "gmx_infiles/conf.gro"
top = "gmx_infiles/topol_amber99sbildn.top"
ndx = "gmx_infiles/index.ndx"
mdp = asyncgmx.MDP("gmx_infiles/md.mdp")

gmx_engine_kwargs = {"mdconfig": mdp,
                     "gro_file": gro,
                     "top_file": top,
                     "ndx_file": ndx,
                     "output_traj_type": "XTC",
                     #"mdrun_extra_args": "-nt 2",
                     # use this for gmx sans (thread) MPI
                     "mdrun_extra_args": "-ntomp 2",
                     }
gmx_engine_cls = asyncgmx.GmxEngine

In [11]:
# state functions
from state_funcs_mda import alpha_R, C7_eq

wrapped_alphaR = asyncmd.trajectory.PyTrajectoryFunctionWrapper(alpha_R)

wrapped_C7_eq = asyncmd.trajectory.PyTrajectoryFunctionWrapper(C7_eq)

In [12]:
# descriptor_transform for the model

# internal coordinates
from state_funcs_mda import descriptor_func_ic

wrapped_transform = asyncmd.trajectory.PyTrajectoryFunctionWrapper(descriptor_func_ic, call_kwargs={"molecule_selection": "protein"})

In [13]:
# load initial transitions
tp_lb = Trajectory(structure_file="gmx_infiles/ala_300K_amber99sb-ildn.tpr", trajectory_files="gmx_infiles/TP_low_barrier_300K_amber99sbildn.trr")
tp_short = Trajectory(structure_file="gmx_infiles/ala_300K_amber99sb-ildn.tpr", trajectory_files="gmx_infiles/TP_short_low_barrier_300K_amber99sbildn.trr")
tp_short2 = Trajectory(structure_file="gmx_infiles/ala_300K_amber99sb-ildn.tpr", trajectory_files="gmx_infiles/TP_short2_low_barrier_300K_amber99sbildn.trr")

In [14]:
print([len(tp) for tp in [tp_lb, tp_short, tp_short2]])

[102, 18, 28]


In [15]:
# get the descriptors for one of them to infer the number of inputs for our model
descriptors_for_tp = await wrapped_transform(tp_lb)
descriptors_for_tp_short = await wrapped_transform(tp_short)
descriptors_for_tp_short2 = await wrapped_transform(tp_short2)

In [16]:
await wrapped_alphaR(tp_short)

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [17]:
await wrapped_C7_eq(tp_short)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True])

In [18]:
# imports for model
import torch.nn.functional as F
import torch

In [19]:
# model definition
n_lay_pyramid = 5
n_unit_top = 10
dropout_base = 0.3
n_unit_base = cv_ndim = descriptors_for_tp.shape[1]
fact = (n_unit_top / n_unit_base)**(1./(n_lay_pyramid))

modules = []

for i in range(1, n_lay_pyramid + 1):
    #print(f"ResUnit {i} is {max(n_unit_top, int(n_unit_base * fact**(i-1)))} units wide.")
    #modules += [aimmd.pytorch.networks.ResNet(n_units=max(n_unit_top, int(n_unit_base * fact**(i-1))),
    #                                          n_blocks=1)
    #            ]

    modules += [aimmd.pytorch.networks.FFNet(n_in=max(n_unit_top, int(n_unit_base * fact**(i-1))),
                                             n_hidden=[max(n_unit_top, int(n_unit_base * fact**i))],  # 1 hidden layer network
                                             activation=torch.nn.Identity(),
                                             dropout={"0": dropout_base * fact**i}
                                             )
                ]
    print(f"ResUnit {i} is {max(n_unit_top, int(n_unit_base * fact**(i)))} units wide.")
    print(f"Dropout before it is {dropout_base * fact**i}.")
    modules += [aimmd.pytorch.networks.ResNet(n_units=max(n_unit_top, int(n_unit_base * fact**i)),
                                              n_blocks=1)
                ]

torch_model = aimmd.pytorch.networks.ModuleStack(n_out=1,  # using a single output we will predict only p_B and use a binomial loss
                                                           # we could have also used n_out=n_states to use a multinomial loss and predict all states,
                                                           # but this is probably only worthwhile if n_states > 2 as it would increase the number of free parameters in the NN
                                                 modules=modules,  # modules is a list of initialized torch.nn.Modules from arcd.pytorch.networks
                                                 )

# move model to GPU if CUDA is available
if torch.cuda.is_available():
    torch_model = torch_model.to('cuda')

# choose and initialize an optimizer to train the model
optimizer = torch.optim.Adam(torch_model.parameters(), lr=1e-3)

ResUnit 1 is 66 units wide.
Dropout before it is 0.18674307214231128.
ResUnit 2 is 41 units wide.
Dropout before it is 0.1162432499771616.
ResUnit 3 is 25 units wide.
Dropout before it is 0.07235873872180604.
ResUnit 4 is 16 units wide.
Dropout before it is 0.045041643884176266.
ResUnit 5 is 10 units wide.
Dropout before it is 0.02803738317757008.


In [20]:
# we take an ExpectedEfficiencyPytorchRCModel,
# this RCmodel scales the learning rate by the expected efficiency factor (1 - n_TP_true / n_TP_expected)**2
model = aimmd.pytorch.EEScalePytorchRCModelAsync(nnet=torch_model,
                                                 optimizer=optimizer,
                                                 states=[wrapped_C7_eq, wrapped_alphaR],
                                                 ee_params={'lr_0': 1e-3,  
                                                            #'lr_min': 5e-5,  # lr_min = lr_0 / 20 is a good choice empirically
                                                            'lr_min': 5e-5,
                                                            'epochs_per_train': 3, # try 5, [10 and 15] next
                                                            #'epochs_per_train': 5,
                                                            'interval': 5,
                                                            #'interval': 10,
                                                            #'window': 75,
                                                            'window': 100,
                                                            'batch_size': 8192,
                                                           },
                                                 descriptor_transform=wrapped_transform,
                                                 cache_file=storage,
                                                 )

In [21]:
# we could use a list with initialized movers
#movers = [[aimmdd.TwoWayShootingPathMover(states=[wrapped_C7_eq, wrapped_alphaR],
#                                          engine_cls=gmx_engine_cls,
#                                          engine_kwargs=gmx_engine_kwargs,
#                                          engine_config=mdp,
#                                          walltime_per_part=0.01,
#                                          T=mdp["ref-t"][0],
#                                         )
#           ] for i in range(n_samplers)
#         ]

# it is easier though to use the `Brain.chains_from_moverlist()` function
# this function will create n-chain identical PathSamplingChains where the movers for each chain are
# specified by movers_cls (a list of mover classes) and movers_kwargs (a dict with keyword arguments used for initialization of the movers)
movers_cls = [aimmdd.pathmovers.TwoWayShootingPathMover]
movers_kwargs = [{'states': [wrapped_alphaR, wrapped_C7_eq],
                  'engine_cls': gmx_engine_cls,
                  'engine_kwargs': gmx_engine_kwargs,
                  # NOTE: choose this as short as possible!
                  #       since ala is super-small and commits fast we should make sure
                  #       that most trials reach a state in the first part
                  #       this in turn makes sure that we do not call gromasc multiple times per trial (saving setup time)
                  #       but still ensures that the resulting trajectories are not too long and large
                  #       it also reduces the time needed per step (we need at least walltime_per_part hours per step)
                  #'walltime_per_part': 0.000015625,  # 0.055125 s per part
                  'walltime_per_part': 0.00003125,  # 0.1125 s per part
                  #'walltime_per_part': 0.0000625,  # 0.225 s per part
                  #'walltime_per_part': 0.000125,  # 0.45 s per part
                  #'walltime_per_part': 0.00025,  # 0.9 s per part
                  #'walltime_per_part': 0.0005,  # 1.8 s per part
                  #'walltime_per_part': 0.001,  # 3.6 s per part
                  #'walltime_per_part': 0.002,  # 7.2 s per part
                  #'walltime_per_part': 0.003,  # 10.8 s per part
                  #'walltime_per_part': 0.004,  # 14.4 s per part
                  'T': mdp["ref-t"][0],
                  "sp_selector": aimmdd.spselectors.RCModelSPSelectorFromTraj(),  # can be used to customize SP-selection params 
                  "max_steps": 500 * 10**5,  # 500 steps * dt (2 fs) = 1 ps
                  }
                 ]

In [22]:
trainset = aimmd.TrainSet(n_states=2)

In [23]:
tasks = [aimmdd.pathsampling.TrainingTask(model=model, trainset=trainset),
         aimmdd.pathsampling.SaveTask(storage=storage, model=model, trainset=trainset),
         aimmdd.pathsampling.DensityCollectionTask(model=model,
                                                   first_collection=100,
                                                   recreate_interval=250,
                                                   interval=10
                                                   ),
         ]

In [24]:
# this would be the full __init__ call to the brain
# it gives you full flexibility of setting up every PathSamplingChain individually
#brain = aimmdd.Brain(model=model, workdir=workdir, storage=storage, movers=movers, mover_weights=[[1.], [1.], [1.]], tasks=tasks)

# this is the 'easy' way
brain = aimmdd.Brain.samplers_from_moverlist(model=model, workdir=workdir, storage=storage,
                                             n_sampler=n_samplers,
                                             movers_cls=movers_cls, movers_kwargs=movers_kwargs,
                                             samplers_use_same_stepcollection=False,
                                             tasks=tasks)
                                           # Note that we left mover_weights=None at its default, this results
                                           # in uniform weights for all movers

In [25]:
# NOTE: old cumbersome setup

#initial_step = aimmdd.logic.MCstep(mover=None, stepnum=0, directory="gmx_infiles", path=tp_short, accepted=True)
#initial_step2 = aimmdd.logic.MCstep(mover=None, stepnum=0, directory="gmx_infiles", path=tp_long, accepted=True)

# take two different initial TPs
#for i, c in enumerate(brain.chains):
#    if i == 2:
#        c.current_step = initial_step2
#        c.chainstore.append(initial_step2)  # save the initial step as first step of every chain
#    else:
#        c.current_step = initial_step
#        c.chainstore.append(initial_step)  # save the initial step as first step of every chain


# NEW: use the new seed_initial_paths() method!
# have a look at before
for i, s in enumerate(brain.samplers):
    print(f"Sampler {i}: ", s.current_step)
    print()
# seed them
brain.seed_initial_paths(trajectories=[tp_lb, tp_short, tp_short2], weights=[1., 1., 1.])
# have a look again
for i, s in enumerate(brain.samplers):
    print(f"Sampler {i}: ")
    print(f"{s.current_step}, with path {s.current_step.path}.")
    print()

Sampler 0:  None

Sampler 1:  None

Sampler 2:  None

Sampler 3:  None

Sampler 4:  None

Sampler 0: 
MCStep(mover=None, stepnum=0, states_reached=None, accepted=True, p_acc=1, predicted_committors_sp=None, weight=1,
       directory=/home/tb/hejung/Documents/sources/aimmd/examples/distributed/gmx_infiles), with path Trajectory(trajectory_files=/home/tb/hejung/Documents/sources/aimmd/examples/distributed/gmx_infiles/TP_low_barrier_300K_amber99sbildn.trr, structure_file=/home/tb/hejung/Documents/sources/aimmd/examples/distributed/gmx_infiles/ala_300K_amber99sb-ildn.tpr).

Sampler 1: 
MCStep(mover=None, stepnum=0, states_reached=None, accepted=True, p_acc=1, predicted_committors_sp=None, weight=1,
       directory=/home/tb/hejung/Documents/sources/aimmd/examples/distributed/gmx_infiles), with path Trajectory(trajectory_files=/home/tb/hejung/Documents/sources/aimmd/examples/distributed/gmx_infiles/TP_short_low_barrier_300K_amber99sbildn.trr, structure_file=/home/tb/hejung/Documents/source

In [26]:
import time

In [27]:
n_steps = 10000
start = time.time()

await brain.run_for_n_steps(n_steps)
#await brain.run_for_n_steps(2000)
#await brain.run_for_n_accepts(25)

end = time.time()
print(f"Running for {n_steps} cummulative MCSteps took {end-start} s (= {(end-start)/60} min).")

Running for 10000 cummulative MCSteps took 18214.41458582878 s (= 303.5735764304797 min).


In [28]:
brain.total_steps

10000

## Save the last model, trainset and brain to storage
This enables us to do the analysis in a different notebook and/or continue the TPS simulation from the last step easily.

In [64]:
# save the last model
storage.rcmodels["model_to_continue_with"] = model
storage.save_trainset(trainset)
storage.save_brain(brain)
# TODO: explain that the SaveTask saves everything too (but only at the specified intervals, os best to save yourself at the end to be sure)

In [67]:
storage.close()